### Segmenting and Clustering Neighborhoods in Toronto

In [188]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

### Creating a instance of the data URL

In [189]:
base_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [190]:
with urlopen(base_url) as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

#### Getting the header of the table

In [191]:
match = soup.find('table', class_='wikitable sortable')

csvData = []
csvHeader = []
csvBodyItem = []

for header in match.find_all('th'):
    csvHeader.append(header.text); 
    
print(csvHeader)


['Postcode', 'Borough', 'Neighbourhood\n']


#### Creating a CSV file and save the values

In [192]:
csv_file = open('ListPostalCodesCanada.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(csvHeader)    

for body in match.find_all('tr'):    
    for itemBody in body.find_all('td'):
        csvBodyItem.append(itemBody.text)

    csv_writer.writerow(csvBodyItem)
    csvBodyItem = []
csv_file.close()

#### Convert the CSV file in pandas table

In [320]:
listPostCode = pd.read_csv("ListPostalCodesCanada.csv", delimiter = ',')
filterNotAss = listPostCode['Borough'] != 'Not assigned'
finalList = listPostCode[filterNotAss]
finalList = finalList.groupby(['Postcode', 'Borough'])['Neighbourhood\n'].agg([('Neighbourhood\n', ', '.join)]).reset_index()
finalList = finalList.replace('\n','',regex=True)


#### Listing the header of the pandas dataframe 

finalList.head()

#### Shape the data frame 

In [318]:
finalList.shape

(103, 3)

#### Get the coordinations

In [331]:
url="http://cocl.us/Geospatial_data"
coordinates = pd.read_csv(url)


In [333]:
coordinates.columns = ['Postcode', 'Latitude', 'Longitude']
coordinates.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge the two dataframes

In [336]:
result = pd.merge(finalList, coordinates, on='Postcode')
result.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
